# Linear Systems

Solving linear systems is arguably one of the most fundamental computation in numerics and statistics. Many problems can be reduced to the solution of one or many (potentially huge) linear systems.

In [1]:
import numpy as np
from probnum.linalg import problinsolve
from probnum.linalg.linear_operators import SymmetricKronecker

# Linear system
A = np.random.rand(10, 10)
A = A.dot(A.T) + 10 * np.eye(10) # Symmetrize and make diagonally dominant
b = np.random.rand(10, 1)

## Prior Information

In [2]:
# Prior distribution on A
covA = SymmetricKronecker(A=np.eye(10), B=np.eye(10))
A = RandomVariable(shape=A.shape, 
                   dtype=A.dtype, 
                   distribution=Normal(mean=A, cov=covA))

/home/j/.virtualenvs/probnum/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: LinearOperator subclass should implement at least one of _matvec and _matmat.
  


NotImplementedError: 

## Probabilistic Linear Solvers

In [ ]:
# Probabilistic linear solver
x, A, Ainv, info = problinsolve(A=A, b=b)

## Solution with Uncertainty

In [ ]:
# Plot resulting distribution on x, A and Ainv